In [1]:
import numpy as np
import os, sys
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import seaborn as sns
from collections import Counter
from tqdm.auto import tqdm, trange
from sklearn.preprocessing import MinMaxScaler
import re
import concurrent.futures
from sklearn.metrics import silhouette_score
from scipy.stats import pearsonr
from collections import OrderedDict
import copy
import pickle
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, DistributedSampler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cloudpickle
from contextlib import contextmanager

from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import CTGANSynthesizer
from sdv.sampling import Condition
from sdv.evaluation.single_table import get_column_plot

import dask.dataframe as dpd
import dask_geopandas as dgpd
from dask.diagnostics import ProgressBar
from dask.distributed import Client

import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

/home/grads/l/lipai.huang/GAN_simulated_events/se_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import gc
gc.collect()

# Coord

In [2]:
coord_gdf = gpd.read_file('../src/coord/coord_gdf.shp')
coord_gdf = coord_gdf.drop(columns=['cell_rmse1', 'cell_r21', 'cell_rmse2', 'cell_r22', 'depth'])

In [3]:
coord_union_gdf = gpd.GeoDataFrame(geometry=[coord_gdf.unary_union], crs=coord_gdf.crs)

# Generated events

In [7]:
syn_events_df = pd.read_parquet('../outputs/aggregated_syn_events.parquet')

In [8]:
syn_events_df

,x,y,cumu_rain,peak_int,duration,channel,ter,syn_depth
0,3.070848e+06,1.395185e+07,2.028108,0.404348,14,0,128.006729,1.649017
1,3.138509e+06,1.393107e+07,0.814155,1.388835,3,0,79.065697,1.743952
2,3.138369e+06,1.393059e+07,9.210388,17.113158,3,0,79.065697,2.602597
3,3.030618e+06,1.394084e+07,8.017052,3.207797,5,0,156.468750,1.850432
4,2.938629e+06,1.394260e+07,1.069939,0.235279,2,0,233.210526,1.648227
...,...,...,...,...,...,...,...,...
4368986,3.150696e+06,1.385620e+07,0.850719,1.894187,1,1,18.980492,1.796558
4368987,3.012808e+06,1.385354e+07,10.247778,5.452115,5,1,97.975548,1.565761
4368988,2.934805e+06,1.391727e+07,0.070056,2.654061,3,1,227.769867,0.955412
4368989,3.035107e+06,1.380791e+07,2.751928,2.407199,5,1,70.938553,1.301932
